# LLMs prompts

School of Epidemiology

Futuri Center, Białystok, October 22, 2025 

Wojciech Łaguna
 

## <span style="color: red;">Warning!</span>

The following materials are for data review purposes only. The results of the prompts **should not be used in scientific work under any circumstances**. The prompts should only be used for preliminary data review. Final statistics and data analysis should be performed by a professional statistician. 

## Prerequisites

1. [Anaconda](https://www.anaconda.com/download/success) (preferred) or stanalone [Jupyter Notebook](https://jupyter.org/install) installed

![image](../Images/anaconda_download.png)

2. Jupyter Lab selected (right-up corner of the image)

![image](../Images/jupyter_lab.png)
   
3. Access to **one of the following LLMs** (free or paid version). In some you need to be registered:
   - [Claude](https://claude.ai/login?returnTo=%2F%3F)
   - [ChatGPT](https://chatgpt.com/)
   - [Gemini](https://gemini.google.com/)
   - [GROK](https://grok.com/)

## Data overview

**[Heart Disease Cleveland](https://www.kaggle.com/datasets/ritwikb3/heart-disease-cleveland/data)**

The dataset is the Cleveland Heart Disease dataset taken from the UCI repository. The dataset consists of 303 individuals’ data.


[Licence:CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/)

1. **Age**: Patients Age in years 
2. **Sex**: Gender (Male : 1; Female : 0) 
3. **cp**: Type of chest pain experienced by patient. This term categorized into 4 category.
    - 0 typical angina,
    - 1 atypical angina,
    - 2 non-anginal pain,
    - 3 asymptomatic 
4. **trestbps**: patient's level of blood pressure at resting mode in mm/HG 
5. **chol**: Serum cholesterol in mg/dl 
6. **fbs**: Blood sugar levels on fasting > 120 mg/dl represents as 1 in case of true and 0 as false 
7. **restecg**: Result of electrocardiogram while at rest are represented in 3 distinct values
   - 0 : Normal
   - 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
   - 2: showing probable or definite left ventricular hypertrophyby Estes' criteria 
8. **thalach**: Maximum heart rate achieved 
9. **exang**: Angina induced by exercise 0 depicting NO 1 depicting Yes
10. **oldpeak**: Exercise induced ST-depression in relative with the state of rest 
11. **slope**: ST segment measured in terms of slope during peak exercise
    - 0: up sloping;
    - 1: flat;
    - 2: down sloping
14. **ca**: The number of major vessels (0–3)
15. **thal**: A blood disorder called thalassemia
    - 0: NULL
    - 1: normal blood flow
    - 2: fixed defect (no blood flow in some part of the heart)
    - 3: reversible defect (a blood flow is observed but it is not normal
16. ***target***: It is the target variable which we have to predict. 1 means patient is suffering from heart disease and 0 means patient is normal.

## Data loading

**PROMPT: I need to load a dataset from given address to pandas 'https://raw.githubusercontent.com/wojla/llm_workshop/refs/heads/main/data/Heart_disease_cleveland_new.csv'. Plase assign it to "df" variable**. 

In [ ]:
import pandas as pd

# Load the dataset from the given path
df = pd.read_csv('https://raw.githubusercontent.com/wojla/llm_workshop/refs/heads/main/data/Heart_disease_cleveland_new.csv')

# Display the first few rows
print(df.head())

**PROMPT: Can I see table in a nicer outline?**

In [ ]:
import pandas as pd
from IPython.display import display

df = pd.read_csv('../data/Heart_disease_cleveland_new.csv')

# Pretty table print in terminal
display(df.head(10))

**PROMPT: What types of data do I have in my dataset?**

In [ ]:
# Show basic info about data types
display(df.dtypes.to_frame(name='Data Type').rename_axis('Column Name').reset_index())

## Data cleaning

**PROMPT: I know that "chol" should be a number, not "object." How can I check if this and other columns displaying the "object" type contain a random string?**

In [ ]:
# Loop over columns with type 'object'
invalid_entries = {}

for col in df.select_dtypes(include='object').columns:
    # Try converting to numeric, track invalid (non-convertible) entries
    mask = pd.to_numeric(df[col], errors='coerce').isna() & df[col].notna()
    if mask.any():
        invalid_entries[col] = df.loc[mask, col].unique()

# Display results
if invalid_entries:
    display(pd.DataFrame(
        [(k, ', '.join(map(str, v))) for k, v in invalid_entries.items()],
        columns=['Column', 'Invalid Entries']
    ))
else:
    print("✅ No non-numeric values found in object-type columns.")

**PROMPT: How can I replace "unknown" by an empty value?**

In [ ]:
import numpy as np
df.replace("unknown", np.nan, inplace=True)

**PROMPT: I replaced, but "chol" is still an "object" in my dataframe.** 

In [ ]:
df['chol'] = pd.to_numeric(df['chol'], errors='coerce')

**PROPMPT: How to discover outliers in my dataset?**

In [ ]:
# Calculate IQR and detect outliers for each numeric column
outlier_summary = []
for col in df.select_dtypes(include=['number']).columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
    outlier_summary.append((col, len(outliers), outliers.min(), outliers.max()))

display(pd.DataFrame(outlier_summary, columns=['Column', 'Outlier Count', 'Min Outlier', 'Max Outlier']))

In [ ]:
import matplotlib.pyplot as plt

df.select_dtypes(include=['number']).boxplot(figsize=(12,6))
plt.title('Boxplots for Numeric Features (IQR Outlier Detection)')
plt.show()

**PROMPT: I see that the maximum values ​​for "chol" and "thalach" are outliers, as are the minimum values ​​for "target." How do I replace them with an empty value?**

In [ ]:
# For 'chol' — replace the maximum value with NaN
df.loc[df['chol'] == df['chol'].max(), 'chol'] = np.nan

# For 'thalach' — replace the maximum value with NaN
df.loc[df['thalach'] == df['thalach'].max(), 'thalach'] = np.nan

# For 'target' — replace the minimum value with NaN
df.loc[df['target'] == df['target'].min(), 'target'] = np.nan

**PROMPT: How many missing values are in my dataset?**

In [ ]:
# Count missing values per column
missing_summary = (
    df.isna().sum()
    .to_frame(name='Missing Values')
    .rename_axis('Column Name')
    .reset_index()
)

# Display as a nice table
display(missing_summary)

# Optionally, total number of missing values in the dataset
print(f"\nTotal missing values in dataset: {df.isna().sum().sum()}")

**PROMPT: How to remove all records that contain at least one empty value?**

In [ ]:
df = df.dropna()
print((df.isna().any(axis=1)).sum())

In [ ]:
df.info()

**PROMPT: How to check if there any duplicates?**

In [ ]:
# Count how many duplicate rows exist
duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

## Basic statistics

**PROMPT: How to obtain column names containing continuous and categorical values?**

In [ ]:
# Identify numeric columns
numeric_cols = df.select_dtypes(include=['number']).columns.tolist()

# Identify categorical columns
categorical_cols = df.select_dtypes(exclude=['number']).columns.tolist()

# Separate continuous vs discrete numeric (optional refinement)
# Continuous: many unique numeric values
# Discrete: few unique numeric values (e.g., 0/1 target or small category codes)
continuous_cols = [col for col in numeric_cols if df[col].nunique() > 10]
discrete_cols = [col for col in numeric_cols if df[col].nunique() <= 10]

# Display results
display(pd.DataFrame({
    'Continuous Columns': pd.Series(continuous_cols),
    'Discrete/Categorical Columns': pd.Series(categorical_cols + discrete_cols)
}))
print("continuous_cols": continuous_cols)
print("discrete_cols": discrete_cols)

**PROMPT: How to display histograms for continuous data?**

In [ ]:
import matplotlib.pyplot as plt

# Identify continuous columns (if not already done)
continuous_cols = [col for col in df.select_dtypes(include='number').columns if df[col].nunique() > 10]

# Plot histograms
for col in continuous_cols:
    plt.figure(figsize=(6, 4))
    plt.hist(df[col].dropna(), bins=30)  # drop NaNs just in case
    plt.title(f'Histogram of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

**PROMPT: How to get basic statistic for continuous data?**

In [ ]:
# Get basic statistics for continuous data
stats = df[continuous_cols].describe().T  # transpose for easier reading
display(stats)

**PROMPT: How to visualize columns with categorical values?**

In [ ]:
# Identify discrete/categorical columns
categorical_cols = [col for col in df.columns if df[col].dtype == 'object' or df[col].nunique() <= 10]

# Plot bar charts
for col in categorical_cols:
    plt.figure(figsize=(6, 4))
    df[col].value_counts().plot(kind='bar')
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.grid(axis='y')
    plt.show()

In [ ]:
df['target'].value_counts().plot(kind='pie', autopct='%1.1f%%', figsize=(5,5))
plt.title('Target distribution')
plt.ylabel('')
plt.show()

**PROMPT: How to get basic statistics for categorical variables?**

In [ ]:
categorical_cols = [col for col in df.columns if df[col].dtype == 'object' or df[col].nunique() <= 10]
p_value
# Compute basic stats for each categorical column
cat_stats = pd.DataFrame({
    'Unique Values': [df[col].nunique() for col in categorical_cols],
    'Most Frequent': [df[col].mode()[0] if not df[col].mode().empty else None for col in categorical_cols],
    'Frequency of Most Frequent': [df[col].value_counts().iloc[0] if not df[col].value_counts().empty else 0 for col in categorical_cols],
    'Missing Values': [df[col].isna().sum() for col in categorical_cols]
}, index=categorical_cols)

display(cat_stats)

**PROMPT: Can I get a table showing the numbers and percentages for each categorical variable?**

In [ ]:
# Build a summary table
cat_summary = []

for col in categorical_cols:
    counts = df[col].value_counts(dropna=False)
    percentages = (counts / len(df) * 100).round(2)
    temp_df = pd.DataFrame({'Category': counts.index, 'Count': counts.values, 'Percentage': percentages.values})
    temp_df.insert(0, 'Variable', col)
    cat_summary.append(temp_df)

# Combine all summaries into one table
cat_summary_df = pd.concat(cat_summary, ignore_index=True)

display(cat_summary_df)

**PROMPT: Can I see correlation matrix for my data?**

In [ ]:
# Compute correlation for numeric columns only
corr_matrix = df.select_dtypes(include='number').corr()

# Display neatly
display(corr_matrix.round(2))

In [ ]:
corr_matrix = df.select_dtypes(include='number').corr()

# Plot heatmap with values
plt.figure(figsize=(10, 8))
plt.imshow(corr_matrix, cmap='coolwarm', interpolation='nearest')
plt.colorbar(label='Correlation')

# Axis ticks
plt.xticks(ticks=np.arange(len(corr_matrix.columns)), labels=corr_matrix.columns, rotation=90)
plt.yticks(ticks=np.arange(len(corr_matrix.columns)), labels=corr_matrix.columns)
plt.title('Correlation Matrix Heatmap')

# Add text annotations for correlation values
for i in range(len(corr_matrix.columns)):
    for j in range(len(corr_matrix.columns)):
        plt.text(j, i, f"{corr_matrix.iloc[i, j]:.1f}",
                 ha='center', va='center', color='black', fontsize=9)

plt.tight_layout()
plt.show()

**PROMPT: I do not see "sex" in the list (Male, Female), how to include them?**

In [ ]:
# Create a numeric version for correlation
df['sex_num'] = df['sex'].map({'Female': 0, 'Male': 1})

## Statistics on groups

**PROMPT: Can you show basic statistics per "sex"?**

In [ ]:
# Compute statistics grouped by 'sex'
sex_stats = df.groupby('sex').describe().T  # Transpose for readability
display(sex_stats)
display(df.groupby('sex').agg(['mean', 'std', 'min', 'max']).T)

In [ ]:
import matplotlib.pyplot as plt
p_value
df.boxplot(column='chol', by='sex', figsize=(6,4))
plt.title('Cholesterol Levels by Sex')
plt.suptitle('')
plt.xlabel('Sex')
plt.ylabel('Cholesterol')
plt.show()

**PROMPT: Can you show how many Men have "chol" level above the median for whole dataset?**

In [ ]:
# Dataset-wide median
chol_median = df['chol'].median()

# Boolean mask for men
men_mask = df['sex'].eq('Male')

# Count men with chol > median (NaNs are excluded)
men_above_median = (men_mask & df['chol'].notna() & df['chol'].gt(chol_median)).sum()

print(f"Median chol (entire dataset): {chol_median:.2f}")
print(f"Men with chol > median: {int(men_above_median)}")

**PROMPT: Show me records with Man who has "chol" levels above 200**

In [ ]:
mask = df['sex'].eq('Male') & (pd.to_numeric(df['chol'], errors='coerce') > 200)

men_high_chol = df.loc[mask].copy()

men_high_chol = men_high_chol.sort_values('chol', ascending=False)
men_high_chol

**PROMPT: How to save selected Men to a csv file?**

In [ ]:
out_path = "men_high_chol.csv"  # change path if needed
men_high_chol.to_csv(out_path, index=False)

print(f"Saved {len(men_high_chol)} rows to {out_path}")

**PROMPT: Can you show a plot of "chol" vs "age" regarding "target"?**

In [ ]:
plt.figure(figsize=(7, 5))

# Scatter plot: color by target (e.g., 0 = no disease, 1 = disease)
for t in df['target'].unique():
    subset = df[df['target'] == t]
    plt.scatter(subset['age'], subset['chol'], label=f'Target {t}', alpha=0.7)

plt.title('Cholesterol vs Age by Target')
plt.xlabel('Age')
plt.ylabel('Cholesterol (chol)')
plt.legend(title='Target')
plt.grid(True)
plt.show()

In [ ]:
from scipy.stats import linregress

plt.figure(figsize=(8, 6))

# Define target labels for readability
target_labels = {0: 'No Disease', 1: 'Disease'}

# Compute and plot regression for each target group
for t, label in target_labels.items():
    subset = df[df['target'] == t]
    x = subset['age']
    y = subset['chol']

    # Perform linear regression
    slope, intercept, r_value, p_value, std_err = linregress(x, y)

    # Scatter plot for this group
    plt.scatter(x, y, alpha=0.6, label=f'{label} (n={len(subset)})')

    # Regression line
    plt.plot(x, slope * x + intercept, linewidth=2)

    # Print regression statistics
    print(f"{label}: slope={slope:.2f}, intercept={intercept:.2f}, R²={r_value**2:.3f}, p={p_value:.3f}")

plt.title('Cholesterol vs Age by Target with Regression Lines')
plt.xlabel('Age')
plt.ylabel('Cholesterol (chol)')
plt.legend(title='Target')
plt.grid(True)
plt.show()

## Statistical tests

**PROMPT: Can you show me how to check if there is statistical significance between men and women for each variable?**

In [ ]:
from scipy.stats import ttest_ind
from IPython.display import display
import pandas as pd

# Identify continuous columns
continuous_cols = [col for col in df.select_dtypes(include='number').columns if df[col].nunique() > 10]

# Split data by sex
male_df = df[df['sex'] == 'Male']
female_df = df[df['sex'] == 'Female']

# Run independent t-tests
t_results = []
for col in continuous_cols:
    t_stat, p_val = ttest_ind(male_df[col].dropna(), female_df[col].dropna(), equal_var=False)  # Welch’s t-test
    t_results.append({'Variable': col, 't-statistic': t_stat, 'p-value': p_val})

t_results_df = pd.DataFrame(t_results)
t_results_df['Significant (p<0.05)'] = t_results_df['p-value'] < 0.05
display(t_results_df)

In [ ]:
from scipy.stats import chi2_contingency

# Identify categorical/discrete columns
categorical_cols = [col for col in df.columns if df[col].dtype == 'object' or df[col].nunique() <= 10]
categorical_cols = [c for c in categorical_cols if c != 'sex']  # exclude sex itself

chi_results = []
for col in categorical_cols:
    contingency = pd.crosstab(df['sex'], df[col])
    chi2, p_val, dof, exp = chi2_contingency(contingency)
    chi_results.append({'Variable': col, 'Chi²': chi2, 'p-value': p_val})

chi_results_df = pd.DataFrame(chi_results)
chi_results_df['Significant (p<0.05)'] = chi_results_df['p-value'] < 0.05
display(chi_results_df)

**PROMPT: How to create two tables containing general statistics: total, men, women. Number and percentage for categorical variables, median and IQR range for continuous variables? Calculate the “p-value” in each table. All floating point numbers should be rounded to three decimal places.**

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, mannwhitneyu
from IPython.display import display

# -----------------------
# Identify variable types
# -----------------------
categorical_cols = [col for col in df.columns if df[col].dtype == 'object' or df[col].nunique() <= 10]
categorical_cols = [c for c in categorical_cols if c != 'sex']  # exclude sex itself
continuous_cols = [col for col in df.select_dtypes(include='number').columns if df[col].nunique() > 10]

# Split data by sex
male_df = df[df['sex'] == 'Male']
female_df = df[df['sex'] == 'Female']

# ========================
# Table 1 — Categorical
# ========================
cat_summary = []
for col in categorical_cols:
    counts = df[col].value_counts(dropna=False)
    total_count = counts.sum()

    # Chi-square test
    contingency = pd.crosstab(df['sex'], df[col])
    chi2, p_val, dof, exp = chi2_contingency(contingency)

    for category in counts.index:
        n_total = counts[category]
        n_male = male_df[col].value_counts(dropna=False).get(category, 0)
        n_female = female_df[col].value_counts(dropna=False).get(category, 0)

        cat_summary.append({
            'Variable': col,
            'Category': category,
            'Total (n,%)': f"{n_total} ({n_total/total_count*100:.3f}%)",
            'Men (n,%)': f"{n_male} ({n_male/len(male_df)*100:.3f}%)",
            'Women (n,%)': f"{n_female} ({n_female/len(female_df)*100:.3f}%)",
            'p-value': round(p_val, 3)
        })

cat_summary_df = pd.DataFrame(cat_summary)
display(cat_summary_df)

# ========================
# Table 2 — Continuous
# ========================
cont_summary = []
for col in continuous_cols:
    # Calculate median and IQR
    med_total = df[col].median()
    q1_total, q3_total = df[col].quantile([0.25, 0.75])
    med_male = male_df[col].median()
    q1_male, q3_male = male_df[col].quantile([0.25, 0.75])
    med_female = female_df[col].median()
    q1_female, q3_female = female_df[col].quantile([0.25, 0.75])

    # Mann–Whitney U test (non-parametric)
    u_stat, p_val = mannwhitneyu(male_df[col].dropna(), female_df[col].dropna(), alternative='two-sided')

    cont_summary.append({
        'Variable': col,
        'Total (median [IQR])': f"{med_total:.3f} [{q1_total:.3f}–{q3_total:.3f}]",
        'Men (median [IQR])': f"{med_male:.3f} [{q1_male:.3f}–{q3_male:.3f}]",
        'Women (median [IQR])': f"{med_female:.3f} [{q1_female:.3f}–{q3_female:.3f}]",
        'p-value': round(p_val, 3)
    })

cont_summary_df = pd.DataFrame(cont_summary)
display(cont_summary_df)


**PROMPT: How to make a "p" correction? Justify your choice of method.**

In [ ]:
from statsmodels.stats.multitest import multipletests
import pandas as pd
from IPython.display import display

# Before correction, keep only one p-value per categorical variable
unique_pvals = pd.concat([
    cont_summary_df[['Variable', 'p-value']],
    cat_summary_df[['Variable', 'p-value']].drop_duplicates('Variable')
], ignore_index=True)

# Apply Benjamini–Hochberg correction
from statsmodels.stats.multitest import multipletests
reject, pvals_corrected, _, _ = multipletests(unique_pvals['p-value'], method='fdr_bh')

unique_pvals['p-corrected'] = pvals_corrected.round(3)

# Merge back to original tables
cont_summary_df = cont_summary_df.merge(unique_pvals[['Variable', 'p-corrected']], on='Variable', how='left')
cat_summary_df = cat_summary_df.merge(unique_pvals[['Variable', 'p-corrected']], on='Variable', how='left')


display(cont_summary_df)
display(cat_summary_df)

## Discovering risk factors

**PROMPT: How to discover risk factors for my "target"?**

In [ ]:
from scipy.stats import mannwhitneyu, chi2_contingency
import pandas as pd
from IPython.display import display

# Identify variable types
categorical_cols = [col for col in df.columns if df[col].dtype == 'object' or df[col].nunique() <= 10]
categorical_cols = [c for c in categorical_cols if c not in ['target']]  # exclude target
continuous_cols = [col for col in df.select_dtypes(include='number').columns if df[col].nunique() > 10]

results = []

# Continuous variables: Mann–Whitney U test
for col in continuous_cols:
    group1 = df[df['target'] == 0][col].dropna()
    group2 = df[df['target'] == 1][col].dropna()
    stat, p_val = mannwhitneyu(group1, group2, alternative='two-sided')
    results.append({'Variable': col, 'Type': 'Continuous', 'p-value': p_val})

# Categorical variables: Chi-square test
for col in categorical_cols:
    contingency = pd.crosstab(df[col], df['target'])
    chi2, p_val, dof, exp = chi2_contingency(contingency)
    results.append({'Variable': col, 'Type': 'Categorical', 'p-value': p_val})

univariate_df = pd.DataFrame(results)
univariate_df['Significant (p<0.05)'] = univariate_df['p-value'] < 0.05
display(univariate_df.sort_values('p-value'))

In [ ]:
import statsmodels.api as sm

# Encode categorical variables (one-hot encoding)
X = pd.get_dummies(df.drop(columns=['target', 'sex']), drop_first=True)
y = df['target']

# Add constant term for intercept
X = sm.add_constant(X)

# Fit logistic regression
model = sm.Logit(y, X).fit()
print(model.summary())


In [ ]:
odds_ratios = pd.DataFrame({
    'Variable': model.params.index,
    'Odds Ratio': np.exp(model.params.values),
    'p-value': model.pvalues.values
})
display(odds_ratios.sort_values('p-value'))

**PROMPT: How to discover feature importance regarding "target"?**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Encode data
X = pd.get_dummies(df.drop(columns=['target', 'sex']), drop_first=True)
y = df['target']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Feature importance
importances = pd.DataFrame({
    'Variable': X.columns,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

display(importances)

print("AUC:", roc_auc_score(y_test, rf.predict_proba(X_test)[:, 1]))


## BONUS. Advanced prompt (don't expect it will produce expected outcome).

**PROMPT**:

I have a cleaned and prepared Heart Disease Cleveland dataset ("df") (303 patients, 14 clinical variables). Now I need to conduct exploratory data analysis to formulate evidence-based research hypotheses for epidemiological investigation.

Dataset context:
- Cleaned dataset ready for analysis
- Target: heart disease presence (binary)
- 14 clinical predictors including demographics, symptoms, and test results
- Balanced analysis needed for hypothesis generation

Please create Python code that performs comprehensive exploratory analysis to generate research hypotheses:

1. **Univariate Analysis by Heart Disease Status:**
   - **Demographics:**
     * Age distribution: histograms, box plots, statistical tests (t-test/Mann-Whitney)
     * Sex distribution: cross-tabulation, chi-square test
     * Calculate mean age difference between diseased/healthy
     * Assess age-sex interaction patterns
   
   - **Clinical Symptoms:**
     * Chest pain types (cp) distribution by heart disease status
     * Exercise-induced angina (exang) prevalence comparison
     * Statistical significance testing for symptom-disease associations
   
   - **Clinical Measurements:**
     * Blood pressure (trestbps): distribution analysis, clinical cut-points
     * Cholesterol (chol): distribution analysis, clinical categories
     * Maximum heart rate (thalach): age-adjusted analysis
     * Fasting blood sugar (fbs) prevalence comparison
   
   - **Diagnostic Tests:**
     * Resting ECG (restecg) abnormalities by disease status
     * ST depression (oldpeak) patterns and clinical significance
     * ST segment slope (slope) distribution analysis
     * Major vessels (ca) involvement patterns
     * Thalassemia (thal) defect types by disease status

2. **Bivariate Analysis and Correlation:**
   - Create comprehensive correlation matrix for numerical variables
   - Cross-tabulation tables for categorical variables
   - Point-biserial correlations for categorical-numerical pairs
   - Identify strongest associations with heart disease
   - Clinical interpretation of correlation patterns

3. **Advanced Visualization for Hypothesis Generation:**
   - **Age-Sex stratified analysis:**
     * Heart disease prevalence by age groups and sex
     * Age-sex interaction heatmaps
     * Stacked bar charts showing disease patterns
   
   - **Risk Factor Combination Analysis:**
     * Multiple chest pain symptoms (cp + exang)
     * Blood pressure and cholesterol combined effects
     * Age with clinical measurements interactions
   
   - **Diagnostic Test Patterns:**
     * ECG abnormalities with ST depression correlation
     * Vessel involvement (ca) with other cardiac markers
     * Thalassemia types with disease severity indicators

4. **Statistical Evidence Summary for Hypothesis Formulation:**
   - Calculate effect sizes (Cohen's d, Cramer's V, odds ratios)
   - Identify variables with strongest associations (p<0.05, effect size >0.3)
   - Rank variables by predictive potential
   - Document clinical plausibility of observed associations

5. **Generate Formal Research Hypotheses:**
   Based on the exploratory analysis, formulate:
   
   **Primary Hypothesis:**
   - Main research question with strongest evidence
   - Specify direction and magnitude of expected effect
   
   **Secondary Hypotheses (3-4):**
   - Additional hypotheses supported by exploratory findings
   - Include interaction hypotheses if patterns suggest them
   
   **Each hypothesis should specify:**
   - Null hypothesis (H₀)
   - Alternative hypothesis (H₁)
   - Expected effect direction and magnitude
   - Clinical/biological rationale
   - Statistical test to be used for verification
   - Required assumptions for the test

6. **Create Evidence Summary Tables:**
   - Summary table of all associations found (variable, effect size, p-value, clinical interpretation)
   - Risk factor ranking by strength of association
   - Suggested analysis plan for hypothesis testing
   - Power analysis estimates for main hypotheses

Please ensure all analyses are clinically grounded and provide epidemiologically meaningful interpretations. Include detailed visualizations that support hypothesis formation and statistical evidence for each proposed hypothesis.

In [ ]:
# ============================
# EDA + HYPOTHESIS GENERATION PIPELINE FOR CLEVELAND HEART DATA
# Assumes a cleaned pandas DataFrame `df` is already in memory with standard columns.
# Only comments are provided; no prose outside the code.
# ============================

import os
import numpy as np
import pandas as pd
import math
from itertools import product, combinations
from collections import defaultdict

import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.stats.contingency_tables import Table2x2
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.power import TTestIndPower, NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize

# ----------------------------
# CONFIG
# ----------------------------
np.random.seed(42)

TARGET = 'target'  # 0/1
# Expected columns in the Cleveland dataset; adjust if your schema differs
NUM_COLS = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']  # numerical/continuous
CAT_COLS = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']  # categorical/discrete
PLOTS_DIR = 'figures'
RESULTS_DIR = 'results'
ALPHA = 0.05
FDR_Q = 0.05

os.makedirs(PLOTS_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

# ----------------------------
# UTILITIES
# ----------------------------
def ensure_numeric(series):
    """Coerce to numeric for safety."""
    return pd.to_numeric(series, errors='coerce')

def cohen_d_independent(x, y):
    """Cohen's d for two independent samples (Welch-compatible pooled SD)."""
    x = pd.Series(x).dropna()
    y = pd.Series(y).dropna()
    nx, ny = len(x), len(y)
    if nx < 2 or ny < 2:
        return np.nan
    sx, sy = x.std(ddof=1), y.std(ddof=1)
    # Pooled SD (unbiased); using weighted by df
    sp = np.sqrt(((nx - 1) * sx**2 + (ny - 1) * sy**2) / (nx + ny - 2)) if (nx + ny - 2) > 0 else np.nan
    if sp == 0 or np.isnan(sp):
        return np.nan
    return (x.mean() - y.mean()) / sp

def rank_biserial_from_u(u_stat, n1, n2):
    """Rank-biserial correlation from Mann-Whitney U (ranges -1..1)."""
    if n1 == 0 or n2 == 0:
        return np.nan
    return 1 - 2*u_stat/(n1*n2)

def cramers_v_from_table(table):
    """Cramer's V for r x c contingency."""
    chi2, p, dof, _ = stats.chi2_contingency(table, correction=False)
    n = table.values.sum()
    r, c = table.shape
    v = np.sqrt(chi2 / (n * (min(r, c) - 1))) if min(r, c) > 1 else np.nan
    return v, chi2, p, dof

def odds_ratio_2x2(table_2x2):
    """Odds ratio (Fisher exact) for 2x2; Haldane-Anscombe correction if zero cell."""
    arr = table_2x2.to_numpy().astype(float)
    if arr.shape != (2, 2):
        return np.nan, np.nan
    if (arr == 0).any():
        arr = arr + 0.5
    or_val = (arr[0,0] * arr[1,1]) / (arr[0,1] * arr[1,0])
    _, p = stats.fisher_exact(arr)
    return or_val, p

def normality_ok(x, alpha=0.05):
    """Shapiro-Wilk for n between 3 and 5000; returns True if normal."""
    x = pd.Series(x).dropna()
    n = len(x)
    if n < 3:
        return False
    _, p = stats.shapiro(x)
    return p > alpha

def binarize_if_binary(series):
    """Return boolean mask if the series is binary (two unique values)."""
    vals = pd.Series(series).dropna().unique()
    if len(vals) == 2:
        mapping = {vals[0]: 0, vals[1]: 1}
        return pd.Series(series).map(mapping)
    return None

def safe_proportions(table_2x2):
    """Return p1, p2 (proportions) from 2x2 table w.r.t. column=target=1 if columns coded as 0/1."""
    # Expect rows = exposure (0/1), columns = outcome (0/1)
    arr = table_2x2.to_numpy()
    if arr.shape != (2, 2):
        return np.nan, np.nan
    p1 = arr[1,1] / arr[1].sum() if arr[1].sum() > 0 else np.nan
    p0 = arr[0,1] / arr[0].sum() if arr[0].sum() > 0 else np.nan
    return p1, p0

def cut_age_groups(s):
    """Create clinically sensible age groups."""
    return pd.cut(s, bins=[-np.inf, 39, 49, 59, 69, np.inf],
                  labels=['<40', '40-49', '50-59', '60-69', '70+'])

def clinical_bp_cut(s):
    """Hypertension cutpoint per resting SBP >=140 mmHg."""
    return pd.Series(np.where(s >= 140, 'HighBP(≥140)', 'NormalBP(<140)'), index=s.index)

def clinical_chol_cut(s):
    """Total cholesterol clinical categories."""
    cats = pd.cut(s, bins=[-np.inf, 199, 239, np.inf],
                  labels=['Desirable(<200)', 'Borderline(200-239)', 'High(≥240)'])
    return cats

def savefig(path):
    """Save figure with tight layout."""
    plt.tight_layout()
    plt.savefig(path, dpi=120)
    plt.close()

# ----------------------------
# 0) BASIC CHECKS
# ----------------------------
# Ensure required columns exist; will raise if not found
missing_needed = [c for c in [TARGET] + NUM_COLS + CAT_COLS if c not in df.columns]
if missing_needed:
    raise KeyError(f"Missing expected columns: {missing_needed}")

# Coerce numerics
for c in NUM_COLS + [TARGET]:
    df[c] = ensure_numeric(df[c])

# Ensure categorical dtype
for c in CAT_COLS:
    if c in df.columns:
        df[c] = df[c].astype('category')

# ----------------------------
# 1) UNIVARIATE ANALYSIS BY HEART DISEASE STATUS
# ----------------------------

# (a) Demographics: Age distribution, boxplots, tests
fig = plt.figure(figsize=(10, 4))
ax1 = fig.add_subplot(1, 2, 1)
# Histogram by target
for t in sorted(df[TARGET].dropna().unique()):
    ax1.hist(df.loc[df[TARGET] == t, 'age'].dropna(), alpha=0.5, bins=12, label=f"target={int(t)}")
ax1.set_title('Age distribution by heart disease status')
ax1.set_xlabel('Age [years]')
ax1.set_ylabel('Count')
ax1.legend()

ax2 = fig.add_subplot(1, 2, 2)
# Boxplot by target
box_data = [df.loc[df[TARGET] == t, 'age'].dropna().values for t in sorted(df[TARGET].dropna().unique())]
ax2.boxplot(box_data, labels=[f"t={int(t)}" for t in sorted(df[TARGET].dropna().unique())])
ax2.set_title('Age by heart disease status')
ax2.set_ylabel('Age [years]')

savefig(f"{PLOTS_DIR}/age_by_target.png")

# Statistical tests for age
age0 = df.loc[df[TARGET] == 0, 'age'].dropna()
age1 = df.loc[df[TARGET] == 1, 'age'].dropna()
if normality_ok(age0) and normality_ok(age1):
    t_stat, p_age = stats.ttest_ind(age1, age0, equal_var=False)
    test_age = 'Welch t-test'
    eff_age = cohen_d_independent(age1, age0)
    eff_name_age = 'Cohen_d'
else:
    u_stat, p_age = stats.mannwhitneyu(age1, age0, alternative='two-sided')
    test_age = 'Mann-Whitney U'
    eff_age = rank_biserial_from_u(u_stat, len(age1), len(age0))
    eff_name_age = 'RankBiserial'
mean_diff_age = age1.mean() - age0.mean()

# (b) Sex distribution: cross-tab + chi-square
ct_sex = pd.crosstab(df['sex'], df[TARGET])
v_sex, chi2_sex, p_sex, dof_sex = cramers_v_from_table(ct_sex)

# (c) Age-Sex interaction (two-way prevalence table)
age_groups = cut_age_groups(df['age'])
prev_age_sex = pd.crosstab([age_groups, df['sex']], df[TARGET], normalize='index').rename(columns={0:'Prev_noHD',1:'Prev_HD'})
prev_age_sex.to_csv(f"{RESULTS_DIR}/prev_age_sex.csv", index=True)

# (d) Clinical symptoms: cp distribution, exang prevalence, tests
ct_cp = pd.crosstab(df['cp'], df[TARGET])
v_cp, chi2_cp, p_cp, dof_cp = cramers_v_from_table(ct_cp)

ct_exang = pd.crosstab(df['exang'], df[TARGET])
v_exang, chi2_exang, p_exang, dof_exang = cramers_v_from_table(ct_exang)

# (e) Clinical measurements: trestbps, chol, thalach, fbs
# trestbps tests
bp0 = df.loc[df[TARGET] == 0, 'trestbps'].dropna()
bp1 = df.loc[df[TARGET] == 1, 'trestbps'].dropna()
if normality_ok(bp0) and normality_ok(bp1):
    t_bp, p_bp = stats.ttest_ind(bp1, bp0, equal_var=False)
    test_bp = 'Welch t-test'
    eff_bp = cohen_d_independent(bp1, bp0)
    eff_name_bp = 'Cohen_d'
else:
    u_bp, p_bp = stats.mannwhitneyu(bp1, bp0, alternative='two-sided')
    test_bp = 'Mann-Whitney U'
    eff_bp = rank_biserial_from_u(u_bp, len(bp1), len(bp0))
    eff_name_bp = 'RankBiserial'

# chol tests
ch0 = df.loc[df[TARGET] == 0, 'chol'].dropna()
ch1 = df.loc[df[TARGET] == 1, 'chol'].dropna()
if normality_ok(ch0) and normality_ok(ch1):
    t_ch, p_ch = stats.ttest_ind(ch1, ch0, equal_var=False)
    test_ch = 'Welch t-test'
    eff_ch = cohen_d_independent(ch1, ch0)
    eff_name_ch = 'Cohen_d'
else:
    u_ch, p_ch = stats.mannwhitneyu(ch1, ch0, alternative='two-sided')
    test_ch = 'Mann-Whitney U'
    eff_ch = rank_biserial_from_u(u_ch, len(ch1), len(ch0))
    eff_name_ch = 'RankBiserial'

# thalach tests (age-adjusted visualization later)
th0 = df.loc[df[TARGET] == 0, 'thalach'].dropna()
th1 = df.loc[df[TARGET] == 1, 'thalach'].dropna()
if normality_ok(th0) and normality_ok(th1):
    t_th, p_th = stats.ttest_ind(th1, th0, equal_var=False)
    test_th = 'Welch t-test'
    eff_th = cohen_d_independent(th1, th0)
    eff_name_th = 'Cohen_d'
else:
    u_th, p_th = stats.mannwhitneyu(th1, th0, alternative='two-sided')
    test_th = 'Mann-Whitney U'
    eff_th = rank_biserial_from_u(u_th, len(th1), len(th0))
    eff_name_th = 'RankBiserial'

# fbs prevalence comparison
ct_fbs = pd.crosstab(df['fbs'], df[TARGET])
v_fbs, chi2_fbs, p_fbs, dof_fbs = cramers_v_from_table(ct_fbs)

# (f) Diagnostic tests: restecg, oldpeak, slope, ca, thal
ct_rest = pd.crosstab(df['restecg'], df[TARGET]); v_rest, chi2_rest, p_rest, dof_rest = cramers_v_from_table(ct_rest)
op0 = df.loc[df[TARGET]==0, 'oldpeak'].dropna(); op1 = df.loc[df[TARGET]==1, 'oldpeak'].dropna()
if normality_ok(op0) and normality_ok(op1):
    t_op, p_op = stats.ttest_ind(op1, op0, equal_var=False); test_op='Welch t-test'; eff_op=cohen_d_independent(op1,op0); eff_name_op='Cohen_d'
else:
    u_op, p_op = stats.mannwhitneyu(op1, op0, alternative='two-sided'); test_op='Mann-Whitney U'; eff_op=rank_biserial_from_u(u_op,len(op1),len(op0)); eff_name_op='RankBiserial'
ct_slope = pd.crosstab(df['slope'], df[TARGET]); v_slope, chi2_slope, p_slope, dof_slope = cramers_v_from_table(ct_slope)
ct_ca = pd.crosstab(df['ca'], df[TARGET]); v_ca, chi2_ca, p_ca, dof_ca = cramers_v_from_table(ct_ca)
ct_thal = pd.crosstab(df['thal'], df[TARGET]); v_thal, chi2_thal, p_thal, dof_thal = cramers_v_from_table(ct_thal)

# Visuals: clinical cut-points for BP and Chol
fig = plt.figure(figsize=(10,4))
ax = fig.add_subplot(1,2,1)
bp_cat = clinical_bp_cut(df['trestbps'])
prev_bp = pd.crosstab(bp_cat, df[TARGET], normalize='index')
prev_bp.plot(kind='bar', ax=ax, legend=True, rot=0, title='Heart disease prevalence by BP category')
ax.set_ylabel('Proportion')

ax2 = fig.add_subplot(1,2,2)
chol_cat = clinical_chol_cut(df['chol'])
prev_chol = pd.crosstab(chol_cat, df[TARGET], normalize='index')
prev_chol.plot(kind='bar', ax=ax2, legend=True, rot=0, title='Heart disease prevalence by Chol category')
ax2.set_ylabel('Proportion')

savefig(f"{PLOTS_DIR}/prevalence_bp_chol.png")

# ----------------------------
# 2) BIVARIATE + CORRELATION
# ----------------------------
# Numeric-numeric correlation (Pearson + Spearman)
num_df = df[NUM_COLS].apply(pd.to_numeric, errors='coerce')
corr_pearson = num_df.corr(method='pearson')
corr_spearman = num_df.corr(method='spearman')
corr_pearson.to_csv(f"{RESULTS_DIR}/corr_numeric_pearson.csv")
corr_spearman.to_csv(f"{RESULTS_DIR}/corr_numeric_spearman.csv")

plt.figure(figsize=(6,5))
plt.imshow(corr_pearson, interpolation='nearest')
plt.colorbar()
plt.xticks(range(len(NUM_COLS)), NUM_COLS, rotation=45, ha='right')
plt.yticks(range(len(NUM_COLS)), NUM_COLS)
plt.title("Pearson correlation (numeric)")
savefig(f"{PLOTS_DIR}/corr_numeric_pearson.png")

plt.figure(figsize=(6,5))
plt.imshow(corr_spearman, interpolation='nearest')
plt.colorbar()
plt.xticks(range(len(NUM_COLS)), NUM_COLS, rotation=45, ha='right')
plt.yticks(range(len(NUM_COLS)), NUM_COLS)
plt.title("Spearman correlation (numeric)")
savefig(f"{PLOTS_DIR}/corr_numeric_spearman.png")

# Point-biserial correlations: target (binary) vs each numeric
pb_results = []
for c in NUM_COLS:
    x = df[c]
    y = df[TARGET]
    mask = x.notna() & y.notna()
    if mask.sum() >= 3:
        r_pb, p_pb = stats.pointbiserialr(y[mask], x[mask])
        pb_results.append({'var': c, 'pointbiserial_r': r_pb, 'p_value': p_pb, 'n': int(mask.sum())})
pb_df = pd.DataFrame(pb_results).sort_values('p_value')
pb_df.to_csv(f"{RESULTS_DIR}/pointbiserial_target_numeric.csv", index=False)

# Cross-tab for categorical vs target
xtabs = {}
for c in CAT_COLS:
    xtabs[c] = pd.crosstab(df[c], df[TARGET], margins=False)
    xtabs[c].to_csv(f"{RESULTS_DIR}/crosstab_{c}_vs_target.csv")

# ----------------------------
# 3) ADVANCED VISUALIZATION FOR HYPOTHESIS GENERATION
# ----------------------------

# (a) Age-Sex stratified prevalence and heatmap
age_cat = cut_age_groups(df['age'])
prev_table = pd.crosstab([age_cat, df['sex']], df[TARGET], normalize='index').rename(columns={1:'Prev_HD'}).fillna(0)
prev_pivot = prev_table['Prev_HD'].unstack('sex').fillna(0)
plt.figure(figsize=(6,4))
plt.imshow(prev_pivot, aspect='auto')
plt.colorbar(label='Prevalence of heart disease')
plt.xticks(range(prev_pivot.shape[1]), list(prev_pivot.columns))
plt.yticks(range(prev_pivot.shape[0]), list(prev_pivot.index))
plt.title('Heart disease prevalence by Age group and Sex')
savefig(f"{PLOTS_DIR}/prevalence_heatmap_age_sex.png")
prev_pivot.to_csv(f"{RESULTS_DIR}/prevalence_by_age_sex.csv")

# Stacked bar by age groups and sex
stack_df = pd.crosstab([age_cat, df['sex']], df[TARGET], normalize='index')
stack_df.plot(kind='bar', stacked=True, figsize=(9,4), title='Stacked prevalence by Age group & Sex')
plt.ylabel('Proportion')
savefig(f"{PLOTS_DIR}/stacked_prevalence_age_sex.png")

# (b) Risk factor combinations: cp + exang; BP + Chol; Age x Thalach
# cp + exang
comb_cp_ex = pd.crosstab(index=[df['cp'].astype(str), df['exang'].astype(str)], columns=df[TARGET], normalize='index')
comb_cp_ex.to_csv(f"{RESULTS_DIR}/comb_cp_exang_vs_target.csv")
comb_cp_ex.plot(kind='bar', stacked=True, figsize=(10,4), title='Prevalence by cp + exang combinations')
plt.ylabel('Proportion')
savefig(f"{PLOTS_DIR}/comb_cp_exang.png")

# BP + Chol (clinical categories)
bp_cat = clinical_bp_cut(df['trestbps'])
chol_cat = clinical_chol_cut(df['chol'])
comb_bp_chol = pd.crosstab(index=[bp_cat, chol_cat], columns=df[TARGET], normalize='index')
comb_bp_chol.to_csv(f"{RESULTS_DIR}/comb_bp_chol_vs_target.csv")
comb_bp_chol.plot(kind='bar', stacked=True, figsize=(10,4), title='Prevalence by BP + Chol categories')
plt.ylabel('Proportion')
savefig(f"{PLOTS_DIR}/comb_bp_chol.png")

# Age group x thalach (low/high by median)
thalach_cut = pd.Series(np.where(df['thalach'] >= df['thalach'].median(), 'HighThalach', 'LowThalach'))
comb_age_thalach = pd.crosstab(index=[age_cat, thalach_cut], columns=df[TARGET], normalize='index')
comb_age_thalach.to_csv(f"{RESULTS_DIR}/comb_age_thalach_vs_target.csv")
comb_age_thalach.plot(kind='bar', stacked=True, figsize=(10,4), title='Prevalence by Age group x Thalach')
plt.ylabel('Proportion')
savefig(f"{PLOTS_DIR}/comb_age_thalach.png")

# (c) Diagnostic test patterns
# ECG abnormalities vs oldpeak (scatter/jitter)
plt.figure(figsize=(7,4))
x_ecg = df['restecg'].cat.codes
y_oldpeak = df['oldpeak']
plt.scatter(x_ecg + (np.random.rand(len(x_ecg))-0.5)*0.15, y_oldpeak, alpha=0.6)
plt.xticks(sorted(x_ecg.unique()), list(df['restecg'].cat.categories))
plt.xlabel('restecg category')
plt.ylabel('oldpeak (ST depression)')
plt.title('oldpeak by restecg category (jittered)')
savefig(f"{PLOTS_DIR}/oldpeak_by_restecg_scatter.png")

# Vessel involvement (ca) vs prevalence
prev_ca = pd.crosstab(df['ca'], df[TARGET], normalize='index')
prev_ca.plot(kind='bar', stacked=True, rot=0, figsize=(6,4), title='Prevalence by number of major vessels (ca)')
plt.ylabel('Proportion')
savefig(f"{PLOTS_DIR}/prevalence_by_ca.png")

# Thal types vs oldpeak distribution
plt.figure(figsize=(7,4))
data = [df.loc[df['thal'] == k, 'oldpeak'].dropna().values for k in df['thal'].cat.categories]
plt.boxplot(data, labels=list(df['thal'].cat.categories))
plt.ylabel('oldpeak')
plt.title('oldpeak by thal category')
savefig(f"{PLOTS_DIR}/oldpeak_by_thal_box.png")

# ----------------------------
# 4) STATISTICAL EVIDENCE SUMMARY (EFFECT SIZES, FDR, RANKING)
# ----------------------------
results = []

# Continuous vs target loop
for c in NUM_COLS:
    g0 = df.loc[df[TARGET]==0, c].dropna()
    g1 = df.loc[df[TARGET]==1, c].dropna()
    if len(g0) >= 3 and len(g1) >= 3:
        if normality_ok(g0) and normality_ok(g1):
            stat, p = stats.ttest_ind(g1, g0, equal_var=False)
            test = 'Welch t-test'
            eff = cohen_d_independent(g1, g0)
            eff_name = 'Cohen_d'
            direction = 'higher_in_HD' if g1.mean() > g0.mean() else 'lower_in_HD'
        else:
            u, p = stats.mannwhitneyu(g1, g0, alternative='two-sided')
            test = 'Mann-Whitney U'
            eff = rank_biserial_from_u(u, len(g1), len(g0))
            eff_name = 'RankBiserial'
            direction = 'higher_in_HD' if np.median(g1) > np.median(g0) else 'lower_in_HD'
        results.append({'variable': c, 'type': 'numeric', 'test': test, 'stat': stat if test.startswith('Welch') else u,
                        'p_value': p, 'effect_name': eff_name, 'effect_size': eff, 'direction': direction,
                        'n0': len(g0), 'n1': len(g1)})

# Categorical vs target loop
for c in CAT_COLS:
    tab = pd.crosstab(df[c], df[TARGET])
    if tab.values.sum() == 0 or tab.shape[0] < 1 or tab.shape[1] < 1:
        continue
    v, chi2, p, dof = cramers_v_from_table(tab)
    results.append({'variable': c, 'type': 'categorical', 'test': 'Chi-square', 'stat': chi2,
                    'p_value': p, 'effect_name': 'Cramers_V', 'effect_size': v, 'direction': 'n/a',
                    'n0': int(tab[0].sum()) if 0 in tab.columns else np.nan,
                    'n1': int(tab[1].sum()) if 1 in tab.columns else np.nan})
    # Add odds ratio for binary 2x2 only
    if tab.shape == (2,2):
        or_val, p_or = odds_ratio_2x2(tab)
        results.append({'variable': f'{c} (OR 2x2)', 'type': 'categorical', 'test': 'Fisher exact',
                        'stat': np.nan, 'p_value': p_or, 'effect_name': 'OddsRatio', 'effect_size': or_val,
                        'direction': 'OR>1 favors HD', 'n0': int(tab[0].sum()), 'n1': int(tab[1].sum())})

res_df = pd.DataFrame(results)
# FDR within numeric and within categorical families (optional); here global FDR
res_df['p_adj_BH'] = multipletests(res_df['p_value'], method='fdr_bh')[1]
res_df['sig_FDR'] = res_df['p_adj_BH'] < FDR_Q
res_df.to_csv(f"{RESULTS_DIR}/evidence_summary_all.csv", index=False)

# Rank by predictive potential: significant & |effect|>0.3
rank_df = (res_df
           .assign(abs_eff=lambda d: d['effect_size'].abs())
           .sort_values(['sig_FDR','abs_eff','p_adj_BH'], ascending=[False, False, True]))
rank_df.to_csv(f"{RESULTS_DIR}/ranked_variables.csv", index=False)

# ----------------------------
# 5) GENERATE FORMAL RESEARCH HYPOTHESES (PROGRAMMATIC TEMPLATES)
# ----------------------------
# Select top candidates from numeric (by |effect|) and categorical (by Cramer's V / OR)
top_numeric = (res_df[(res_df['type']=='numeric') & res_df['sig_FDR']]
               .sort_values('effect_size', key=np.abs, ascending=False).head(3))
top_categorical = (res_df[(res_df['type']=='categorical') & res_df['sig_FDR'] & (res_df['effect_name']=='Cramers_V')]
                   .sort_values('effect_size', ascending=False).head(3))

hypotheses = []

# Primary hypothesis: choose strongest signal overall
if not rank_df.empty:
    primary = rank_df.iloc[0]
    var = primary['variable']
    eff = primary['effect_size']
    p = primary['p_adj_BH']
    if primary['type'] == 'numeric':
        direction = 'increases' if eff > 0 else 'decreases'
        test_name = 'Welch t-test or Mann-Whitney (as appropriate); logistic regression for confirmation'
        effect_desc = f"expected standardized effect ≈ {eff:.2f}"
    else:
        direction = 'associated with'
        test_name = 'Chi-square (or Fisher for sparse), logistic regression'
        effect_desc = f"expected association strength (Cramer\'s V) ≈ {eff:.2f}"
    hypotheses.append({
        'title': f"Primary: {var} {direction} heart disease risk",
        'H0': f"No association between {var} and heart disease.",
        'H1': f"{var} is {direction} heart disease risk; {effect_desc}.",
        'test': test_name,
        'alpha': ALPHA,
        'FDR_q': FDR_Q,
        'note': f"Selected by strongest effect (FDR-adjusted p={p:.3g})."
    })

# Secondary hypotheses from remaining top signals
for row in pd.concat([top_numeric, top_categorical]).head(4).itertuples():
    var = row.variable
    eff = row.effect_size
    if row.type == 'numeric':
        direction = 'higher in diseased' if row.direction=='higher_in_HD' else 'lower in diseased'
        test_name = 'Welch t-test or Mann-Whitney; confirm via logistic regression with age/sex adjustment'
        effect_desc = f"expected effect size ≈ {eff:.2f}"
    else:
        direction = 'associated with disease status'
        test_name = 'Chi-square/Fisher; confirm via logistic regression'
        effect_desc = f"expected Cramer\'s V ≈ {eff:.2f}"
    hypotheses.append({
        'title': f"Secondary: {var} {direction}",
        'H0': f"No association between {var} and disease status.",
        'H1': f"{var} is {direction}; {effect_desc}.",
        'test': test_name,
        'alpha': ALPHA,
        'FDR_q': FDR_Q,
        'note': "Derived from EDA signals."
    })

# vals: matrix of prevalence values (float), handle possible NaNs safely
vals = np.asarray(prev_pivot.values, dtype=float)

# range across cells (nan-safe); triggers interaction hypothesis if > 0.15
rng = np.nanmax(vals) - np.nanmin(vals) if vals.size else 0.0

if vals.size and np.isfinite(rng) and rng > 0.15:
    hypotheses.append({
        'title': "Secondary (Interaction): Age × Sex modifies heart disease prevalence",
        'H0': "No interaction between age group and sex on heart disease prevalence.",
        'H1': "Prevalence increases with age more steeply in one sex (interaction present).",
        'test': "Logistic regression with AgeGroup, Sex, and AgeGroup×Sex term; LRT for interaction",
        'alpha': ALPHA,
        'FDR_q': FDR_Q,
        'note': "Triggered by heatmap heterogeneity."
    })

# Export hypotheses as a human-readable CSV/text
hypo_df = pd.DataFrame(hypotheses)
hypo_df.to_csv(f"{RESULTS_DIR}/hypotheses_generated.csv", index=False)

# ----------------------------
# 6) EVIDENCE SUMMARY TABLES + POWER ANALYSIS
# ----------------------------
# Assemble a compact summary table with clinical-style interpretation placeholders
summary_rows = []
for row in res_df.itertuples():
    var = row.variable
    eff = row.effect_size
    p_adj = row.p_adj_BH
    interpretable = ''
    if row.type == 'numeric' and not np.isnan(eff):
        interpretable = f"{row.effect_name}={eff:.2f} ({row.direction})"
    elif row.effect_name == 'Cramers_V' and not np.isnan(eff):
        interpretable = f"Cramér's V={eff:.2f} (association)"
    elif row.effect_name == 'OddsRatio' and not np.isnan(eff):
        interpretable = f"OR≈{eff:.2f} (>1 suggests higher odds in exposed)"
    summary_rows.append({
        'variable': var,
        'type': row.type,
        'test_used': row.test,
        'stat_or_effect': interpretable,
        'p_value': row.p_value,
        'p_adj_BH': p_adj,
        'significant_FDR': row.sig_FDR
    })
summary_df = pd.DataFrame(summary_rows).sort_values(['significant_FDR','p_adj_BH'], ascending=[False, True])
summary_df.to_csv(f"{RESULTS_DIR}/evidence_summary_clinical.csv", index=False)

# Rank by strength (significant and |effect|>0.3)
rank_filtered = (res_df.assign(abs_eff=lambda d: d['effect_size'].abs())
                 .query('sig_FDR and abs_eff > 0.3')
                 .sort_values(['abs_eff','p_adj_BH'], ascending=[False, True]))
rank_filtered[['variable','effect_name','effect_size','p_adj_BH']].to_csv(f"{RESULTS_DIR}/risk_factor_ranking.csv", index=False)

# Power analysis for primary numeric and 2x2 hypotheses (rough, based on observed effect sizes)
power_results = []

# Numeric (two-sample t-test) example: use strongest numeric signal
num_sig = res_df[(res_df['type']=='numeric') & res_df['sig_FDR']].copy()
if not num_sig.empty:
    best = num_sig.iloc[num_sig['effect_size'].abs().argmax()]
    d = np.nan_to_num(best['effect_size'])
    # Estimate group sizes from dataset
    n0 = int((df[TARGET]==0).sum())
    n1 = int((df[TARGET]==1).sum())
    ratio = n1 / max(n0,1)
    tpower = TTestIndPower()
    # Required per-group n for 80% power
    try:
        n_per_group = math.ceil(tpower.solve_power(effect_size=abs(d), alpha=ALPHA, power=0.80, ratio=ratio))
    except Exception:
        n_per_group = np.nan
    # Achieved power with current sample
    try:
        achieved = tpower.power(effect_size=abs(d), alpha=ALPHA, nobs1=n0, ratio=ratio)
    except Exception:
        achieved = np.nan
    power_results.append({'hypothesis': f'Numeric: {best.variable}', 'effect': d, 'required_n_per_group_80pct': n_per_group, 'achieved_power': achieved})

# Binary 2x2 example: pick a binary categorical with strongest OR signal if available
or_rows = res_df[res_df['effect_name']=='OddsRatio'].copy()
if not or_rows.empty:
    or_best = or_rows.iloc[or_rows['p_adj_BH'].argmin()]
    # Reconstruct 2x2 and compute p1, p0
    var = or_best['variable'].replace(' (OR 2x2)','')
    tab = pd.crosstab(df[var], df[TARGET])
    if tab.shape == (2,2):
        p1, p0 = safe_proportions(tab)
        if np.isfinite(p1) and np.isfinite(p0):
            es = proportion_effectsize(p1, p0)  # Cohen's h
            npower = NormalIndPower()
            try:
                n_per_group = math.ceil(npower.solve_power(effect_size=abs(es), alpha=ALPHA, power=0.80, ratio=1.0))
            except Exception:
                n_per_group = np.nan
            try:
                achieved = npower.power(effect_size=abs(es), alpha=ALPHA, nobs1=int(tab.iloc[0].sum()), ratio=1.0)
            except Exception:
                achieved = np.nan
            power_results.append({'hypothesis': f'Binary: {var}', 'effect_Cohen_h': es, 'required_n_per_group_80pct': n_per_group, 'achieved_power': achieved})

pd.DataFrame(power_results).to_csv(f"{RESULTS_DIR}/power_analysis_estimates.csv", index=False)

# ----------------------------
# PRINT KEY SUMMARIES (OPTIONAL)
# ----------------------------
print("Saved:")
print(f"- Figures to: {PLOTS_DIR}/")
print(f"- Numeric correlations: {RESULTS_DIR}/corr_numeric_*.csv")
print(f"- Crosstabs: {RESULTS_DIR}/crosstab_*_vs_target.csv")
print(f"- Evidence summary: {RESULTS_DIR}/evidence_summary_all.csv")
print(f"- Ranked variables: {RESULTS_DIR}/ranked_variables.csv")
print(f"- Hypotheses: {RESULTS_DIR}/hypotheses_generated.csv")
print(f"- Power analysis: {RESULTS_DIR}/power_analysis_estimates.csv")

# ----------------------------
# NOTES (in-code, non-executable):
# - Effect-size thresholds: consider |d|≥0.3 (small-moderate), Cramér’s V≥0.3 as meaningful screens for hypotheses.
# - Odds ratios computed only for binary 2x2 tables. For multi-level factors, use logistic regression to estimate per-level OR (ref vs baseline).
# - For thalach age-adjustment beyond visuals, fit a simple logistic regression (target ~ thalach + age) as confirmatory step later.
# - Generated hypotheses are templates filled with observed directions/effects and are meant to guide confirmatory modeling, not serve as final proof.
# ============================
